In [1]:
# Версия с обработчиком мгновенных скоростей и траекторий

import cv2
import os
import fnmatch
import re
import trackpy as tp 
import numpy 
import matplotlib.pylab as plt 
import matplotlib
import pims_nd2
import numpy as np
from matplotlib.widgets import TextBox
import pandas as pd
from PIL import Image
import h5py


In [2]:
# Папка, в которой находится файл


"""папка, в которой лежит файл"""
folder = r'C:\Users\julia\OneDrive\Desktop\SignLab\spherocytosis\18072023\sphero'  


"""название обрабатываемого файла"""
filename = 'sphero003'

# Номера каналов для обсчёта:
    
# Если было снято в каналах 405 (Хёкст), 488 (DiOC6, CD61) и 640 (Annexin-V-Alexa647, P-Selectin-Alexa647)
# 405 - 0
# 488 - 1
# 640 - 2

# Если было снято в каналах 488 (DiOC6, CD61) и 640 (Annexin-V-Alexa647, P-Selectin-Alexa647)
# 488 - 0
# 640 - 1

channel = 1

In [3]:
import cv2
import numpy as np


In [4]:
%matplotlib qt


"""класс, который делает интерфейс для выбора траекторий, соответствующих нейтрофилам, а не грязи тромбоцитов и тд """
class ImageProcessingGUI():
    def __init__(self, img, img_labels, trajectories, dictionary_frames):
        """фигура, на которой демнстрируется текущий слайд""";
        self.fig = plt.figure(figsize=(20,15))
        """видос в виде numpy-массива"""
        self.img = img
        self.img_labels = img_labels
        grid_sz = 15
        """начальный индекс видео"""
        self.currentvideoindice = 1
        
        """trackpy-словарь, с помощью которого рисуются траектории частиц на графике"""
        self.trajectories = trajectories
        """дефолтный тип нейтрофила"""
        self.currtype = 'not specified'
        
        """словарь с информацией о траекториях частиц, другой формат, чем img_labels"""
        self.dictionary_frames = dictionary_frames
        
        self.curr_type_label = ''
        plt.subplot2grid((grid_sz, grid_sz), (0, 0), colspan=grid_sz, rowspan=grid_sz - 1)
        """эта функция из библиотеки trackpy рисует на графике траектории"""
        tp.plot_traj(self.trajectories, label=True)
        
        """показываем на графике начальный кадр видео"""
        self.zerozero = plt.imshow(self.img[self.currentvideoindice], alpha=0.9)
        
        
        """нужно для отображения интерфейса"""
        z_stack_ax = plt.subplot2grid((grid_sz, grid_sz), (grid_sz - 1, grid_sz * 5 // 6))
        z_stack_ax1 = plt.subplot2grid((grid_sz, grid_sz), (grid_sz - 1, grid_sz * 7 // 9))
        
        """лист, в который записываются номера траекторий, соответствующих нейтрофилам"""
        self.selected_cell_labels = []
        """окошко, куда вводятся номера траекторий, соответствующих нейтрофилам"""
        self.CellBox = TextBox(z_stack_ax, 'Leuco Traj N')
        
        """окошко, куда вводятся тип текущей траектории"""
        self.CellBoxType = TextBox(z_stack_ax1, 'Leuco Type')
        self.CellBox.on_submit(self.on_traj_number_update_process)
        
        self.CellBoxType.on_submit(self.on_type_number_update_process)
        
        """убираем оси на картинке, чтобы было красиво"""
        self.zerozero.axes.get_xaxis().set_visible(False)
        self.zerozero.axes.get_yaxis().set_visible(False)
        plt.title('Use sliders to choose current slide')
        plt.axis('off')
        
        
        slider_ax1 = plt.subplot2grid((grid_sz, grid_sz), (grid_sz - 1, grid_sz * 1 // 6))
        button_ax = plt.subplot2grid((grid_sz, grid_sz), (grid_sz - 1, grid_sz * 2 // 6))
        """кнопка завершения обработки текущей части видео"""
        self.done_button = plt.Button(button_ax, 'Done')
        self.done_button.on_clicked(self.on_button_press)
        
        """слайдер для переключения кадров видео"""
        self.cutting = plt.Slider(slider_ax1, 'Current slide', 0, (np.shape(img)[0] - 1),
                                  valinit=self.currentvideoindice, valstep=1)
        self.cutting.on_changed(self.on_slider_process)
        self.enum_particles = 0
        plt.show()
        self.fig.canvas.start_event_loop(timeout=-1)

    def on_button_press(self, event):
        plt.close()
        self.fig.canvas.stop_event_loop()

    def on_slider_process(self, val):
        
        """меняем кадр на видео"""
        self.currentvideoindice = int(val)
        self.zerozero.set_data(self.img[self.currentvideoindice])
        self.zerozero.set_norm(matplotlib.colors.Normalize())
        self.zerozero.set_cmap('viridis')
        
        # self.imgprev.set_data(self.img_labels[self.currentvideoindice])
        plt.draw()

    def on_traj_number_update_process(self, val):
        try:
            
            """сохраняем траекторию, соответствующую нейтрофилу"""
            self.label = np.array(eval(val),ndmin=1)
           
            print(self.label)
            
            """у всех нейтрофилов сквозная нумерация"""
            self.enum_particles+= 1
            #print({'indices':self.label,'celltype':self.currtype} )
            
            """после ввода удаляем значение из бокса, так удобнее вводить"""
            self.CellBox.set_val('')
            self.selected_cell_labels.append({'indices':self.label,'celltype':self.currtype} )
            
        except Exception as e:
            print('Ошибка ввода номера!', e)
            
    def on_type_number_update_process(self, val):
        """вводим тип нейтрофила. тип будет присвоен следующему сохраненному нейтрофилу"""
        try:
            self.currtype =  val
            #self.selected_cell_labels.append(self.label)
        except Exception as e:
            print('Ошибка ввода типа!',e)
"""читаем количество полей в файле нейтрофила"""
def get_z_size(trace):
    
    """читаем файл"""
    frames = pims_nd2.ND2_Reader(trace)
    """смотрим на количество полей в файле"""
    z = frames.sizes.get('m')
    """читаем из метаданных размер пикселя"""
    pixel_size = frames[0].metadata['mpp']
    """закрываем файл данных"""
    frames.close()
    return z, pixel_size
"""смотрим локальное время и дату начала записи файла"""
def GetFilmingInitTime(trace):
    """читаем файл"""
    frames = pims_nd2.ND2_Reader(trace)
    """смотрим на время начала"""
    datetime_init = (frames.metadata['time_start'])
    
    frames.close()
    return datetime_init


"""читаем времена сьемки отдельных кадров"""

def getND2timeframes(z_stack, trace,channel_number=1):
    frames = pims_nd2.ND2_Reader(trace)
    """читаем файл"""
    """если несколько полей, выбираем нужное поле, для которого читаем время тк время может не совпадать для разных полей"""
    if (z_stack):
        frames.default_coords['m'] = z_stack
    """выбираем нужный канал"""    
    frames.default_coords['c'] = channel_number
    """ищем длину файла в кадрах"""
    t = frames.sizes.get('t')
    times = numpy.zeros((t))
    """записываем в массив времена кадров"""
    for i in range(t):
        times[i] = frames[i].metadata['t_ms'] / 1000
    frames.close()    
    return times    
def getNd2efunc(z_stack, trace, channel_number=1, max_chunx = None, split_image = None):
    
    """загружаем файл"""
    frames = pims_nd2.ND2_Reader(trace)
    
    """читаем размер видео"""
    x = frames.sizes.get('x')
    y = frames.sizes.get('y')
    """если в файле много полей, обрабатываем его по полям"""
    if (z_stack):
        frames.default_coords['m'] = z_stack
        
    """выбираем нужный канал"""    
    frames.default_coords['c'] = channel_number
    # делаем массив вместо nd2-файла
    
    """загружаем длину видео в кадрах"""
    t = frames.sizes.get('t')
    times = numpy.zeros((t))
    """если у нас указано, на сколько подполей резать большое поле зрения, режем его"""
    if (split_image!=None):
        """берем корень из числа полей Number, делаем number  x  number субполей"""
        number = (np.sqrt(max_chunx))+1
        one_size_x = int(np.floor(x/number))
        nx = int(split_image%number)
        ny = int(split_image//number)
        one_size_y = int(np.floor(y/number))
        print(nx,ny,one_size_x,one_size_y)
        
        img = numpy.zeros((t, one_size_x, one_size_y))
    else:
        img = numpy.zeros((t, x, y))
    #print(x,y,nx,ny,(nx)*one_size_x,(ny)*one_size_y)    
    for i in range(t):
    
        if (split_image!=None):
            img[i, :, :] = frames[i].T[nx*one_size_x:(nx+1)*one_size_x,ny*one_size_y:(ny+1)*one_size_y]
            
            
        else: 
            img[i, :, :] = frames[i].T
        times[i] = frames[i].metadata['t_ms'] / 1000
        
        
    """сохраняем медианное изображение растущего тромба -- это нужно для другой подзадачи. h5 -- это такой формат файла """    
    
    frames_fon = frames[:]
    frames.close()
        
    if (split_image!=None):    
        return img, times, split_image
    else:
        return img, times, z_stack

In [5]:
def ProcessImage(frames):
    """с помощью библиотеки trackpy делаем треки тромбоцитов"""
    
    frames_fon, times, z_stack = frames
    """tp.batch ищет на всех фреймах обьекты заданного размера. 39 -- размер искомых обьектов в пикселях"""
    f = tp.batch(frames_fon[:], 49)
    """tp.link_df соединяет обьекты в траектории. 15 -- макс смещение в пикселях. memory -- на сколько кадров может пропасть обьект"""
    t = tp.link_df(f, 20, memory=3)
    
    """отфильтровываем слишком короткие траектории"""
    f = tp.filter_stubs(t, 7)
    #drift = tp.compute_drift(f, 3) # Save good drift curves.
    #f = tp.subtract_drift(f, drift)
    dictionary = {}
    """переформатируем данные, которые выдает trackpy, теперь у нас в dictionary хранятся просто отдельные треки"""
    for i in (f['particle']):
        particle_center_list = []
        key = (f[f['particle'] == i])
        frame_list_for_particles = (f[f['particle'] == i])['frame']

        for j in frame_list_for_particles:
            centurx1 = key[key['frame'] == j]
            centurx = np.float64(centurx1['x'])
            century = np.float64(centurx1['y'])
            size = np.float64(centurx1['size'])
            particle_center_list.append([j, centurx, century, size])

        dictionary[i] = particle_center_list
    frame_labels = np.zeros_like(frames)
    # print (dictionary)
    return None, frames_fon, frame_labels, f, dictionary, times, z_stack

In [6]:

TRACE = folder + '/' + filename
z_max, mpp = get_z_size(TRACE + '.nd2')

"""читаем количество полей"""



C:\Users\julia\AppData\Roaming\Python\Python39\site-packages\pims\base_frames.py:478: UserWarning: Please call FramesSequenceND.__init__() at the start of thethe reader initialization.
  warn("Please call FramesSequenceND.__init__() at the start of the"


'читаем количество полей'

In [ ]:
from joblib import Parallel, delayed
"""количество субполей, на которые делятся большие поля зрения, если делить не надо, ставим 0"""
max_chunks = 9
array_of_results = np.zeros(z_max)

"""если у нас несколько полей, раскомментируем эту строчку и комментируем следующие"""
#if (z_max):
    #all_z_stack_results = (Parallel(n_jobs=-1)(delayed(getNd2efunc)(i, TRACE + '.nd2', channel) for i in range(z_max)))
"""параллельно обрабатываем имеющиеся поля или части изображения""" 


img = (Parallel(n_jobs=5)(delayed(getNd2efunc)(0, TRACE + '.nd2', channel,max_chunks, i) for i in range(max_chunks)))


all_z_stack_results = (Parallel(n_jobs=8)(delayed(ProcessImage)(subimage) for subimage in img))
    
zstack_num = z_max

C:\Users\julia\AppData\Local\Temp\ipykernel_15596\1956862355.py:4: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  array_of_results = np.zeros(z_max)


In [ ]:
for all_elems in all_z_stack_results:
    all_coords_array_t = []
    all_coords_array_x = []
    all_coords_array_y = []
    all_coords_array_names = []
    percent = []
    binarize_video, frames_fon, frame_labels, f, dictionary, times, zstack_current_num = all_elems
    gui = ImageProcessingGUI(frames_fon, frame_labels, f, dictionary)
    label_list = gui.selected_cell_labels
    
    all_velocities_array = []
    print(gui.selected_cell_labels)
    for all_neutrophils in gui.selected_cell_labels:
        velocity_array = []
        x_array = []
        y_array = []
        t_array = []
        name_array = []
        for j in (all_neutrophils['indices']):
            #j = int(re.sub("[^0-9]", "", j))
            #velocities = np.zeros(len(dictionary[j]))
            try:
                for frame_enum in range(1, len(dictionary[j])):
                    frame_nr = dictionary[j][frame_enum][0]
                    delta_t = times[frame_nr] - times[dictionary[j][frame_enum - 1][0]]
                    delta_x = dictionary[j][frame_enum][1] - dictionary[j][frame_enum - 1][1]
                    delta_y = dictionary[j][frame_enum][2] - dictionary[j][frame_enum - 1][2]

                    velocity = np.sqrt(delta_x ** 2 + delta_y ** 2)
                    velocity = velocity * mpp / delta_t
                    velocity_array.append(velocity)
                    t_array.append(times[frame_nr])
                    x_array.append(dictionary[j][frame_enum][1])
                    y_array.append(dictionary[j][frame_enum][2])
                    size_leuco = dictionary[j][frame_enum][3]
                    x_this_slide = dictionary[j][frame_enum][1]
                    y_this_slide = dictionary[j][frame_enum][2]
                   
                    name_array.append(all_neutrophils['celltype'])
            except Exception as e:
                print(e)
        all_velocities_array.append(velocity_array)
        all_coords_array_t.append(t_array)
        all_coords_array_x.append(x_array)
        all_coords_array_y.append(y_array)
        all_coords_array_names.append(name_array)

    fig = plt.figure() 
    
    time = times.tolist()
    first = time.index(min(time, key=lambda x: abs(x - 300)))
    fig.suptitle('300 second clot, field #'+str(zstack_current_num+1))
    plt.imshow(frames_fon[first])        
    #plt.imshow(binarize_video[first], alpha = 0.3)
    #plt.show()
    fig = plt.figure()
    fig.suptitle('600 second clot, field #'+str(zstack_current_num+1))
    plt.imshow(frames_fon[-1]) 
    #plt.imshow(binarize_video[-1], alpha = 0.3)
    #plt.show()

    import xlwt

    ex = xlwt.Workbook()
    vel = ex.add_sheet('Velocity', cell_overwrite_ok=True)
    thrombus = ex.add_sheet('Clot', cell_overwrite_ok=True)
    Mean_vals = ex.add_sheet('Mean values', cell_overwrite_ok=True)
    coords = ex.add_sheet('Coords', cell_overwrite_ok=True)
    num = 0
    for different_velocities in (all_velocities_array):
        num += 1
        Mean_vals.write(1,2 * num+4, np.mean(different_velocities))
        Mean_vals.write(0,2*num+4,"Mean Velocity №"+str(num))
        vel.write(0,2*num + 0, 'Velocity №'+str(num))
        for i in range(len(different_velocities)):
            vel.write(i+1, 2*num + 0, different_velocities[i])
    num = 0      
    for different_coords in (all_coords_array_x):
        num += 1
        coords.write(0,4 * num + 0, 'Coords X №'+str(num))
        for i in range(len(different_coords)):
            coords.write(i+1, 4 * num + 0, different_coords[i])
    num = 0        
    for different_coords_y in (all_coords_array_y):
        num += 1
        coords.write(0,4 * num + 1, 'Coords Y №'+str(num))
        for i in range(len(different_coords_y)):
            coords.write(i+1, 4 * num + 1, different_coords_y[i])
    num = 0        
    for different_coords_t in (all_coords_array_t):
        num += 1
        coords.write(0,4 * num +2, 'T №'+str(num))
        for i in range(len(different_coords_t)):
            coords.write(i+1, 4 * num +2, different_coords_t[i])
    num = 0        
    for different_names in (all_coords_array_names):
        num += 1
        coords.write(0,4 * num +3,  'Type №'+str(num))
        vel.write(0,2 * num +1,  'Type №'+str(num))
        Mean_vals.write(0,2 * num+5, 'Type №'+str(num))
        Mean_vals.write(1,2 * num+5, different_names[1])
        for i in range(len(different_names)):
            
            coords.write(i+1, 4 * num + 3, different_names[i])
            vel.write(i+1, 2 * num + 1, different_names[i])
    all_names_for_means = np.unique(all_coords_array_names)          
    num = 0  
    thrombus.write(0, 2, 'Clot square')
    thrombus.write(0, 0, 'Time')
   
    Mean_vals.write(1, 0, '300s')
    Mean_vals.write(1, 1, '600s')
    Mean_vals.write(0, 3, 'Mean velocities')
    ex.save(TRACE + '_field' + str(zstack_current_num + 1) + '.xls')

    plt.show()

### all_leuco_names

Сейчас берем все .excel файлы в папке и находим те, что соответствуют лучше всего 10, 20 и 30 минуте 
Вытаскиваем из них ускорения и длины траекторий 

## 